In [1]:

import pandas as pd
import altair as alt
import pmlb
import warnings
warnings.filterwarnings('ignore')

In [2]:
Indiv_filename = '../Data/4vis/Indiv.pkl'
Indiv_df = pd.read_pickle(Indiv_filename)
dict_method_name_to_attr_name = {
    'shap' : 'shap_value',
    'lime': 'lime_weight', 
    'ig':  'ig_attr', 
    'deepLift':'deepLift_attr' 
}

In [3]:
charts = []
sorted_features = Indiv_df.groupby(['feature_name']).sum().sort_values(by = ['sum_rank'], ascending = False).index.values.tolist()
brush = alt.selection_interval()

In [4]:
for feature in sorted_features:
    charts_per_feature = []
    Attr_feature = Indiv_df[Indiv_df['feature_name'] == feature]

    for method_name, attr_name in dict_method_name_to_attr_name.items():
        Attr_feature[method_name+'_is_positive'] = Attr_feature[attr_name] > 0
    
    
    base = alt.Chart(Attr_feature).mark_circle().encode(
        x=alt.X('feature_value', title = feature + ' value'),
        opacity = alt.condition(brush, alt.value(1.0), alt.value(0.2))
    ).resolve_scale(
        x = "independent",
        y = "independent"
    ).add_selection(
        brush
    )

    for method_name, attr_name in dict_method_name_to_attr_name.items():
        charts_per_feature.append(base.encode(
            y = attr_name,
            color=alt.Color(method_name+'_is_positive', legend=None, scale=alt.Scale(range=['crimson', 'steelblue']))
        )
    )
    charts.append(charts_per_feature)

In [5]:
def View_scatter_feature(feature_index):
    row1 = charts[feature_index][0] | charts[feature_index][1] 
    row2 = charts[feature_index][2] | charts[feature_index][3] 
    return alt.vconcat(row1, row2)

In [6]:
View_scatter_feature(feature_index=0)

alt.VConcatChart(...)